In [101]:
import pandas as pd 
import numpy as np 
from FileOpener.CSVReader.CSVReader import CSVReader as csv
from dateutil.relativedelta import relativedelta

In [102]:
directory = 'CSVFile/Price/Korea/Stocks'

kse_price = csv.price_reader('KOSPI_FROM_1998_close.csv', directory)

In [103]:
from FileOpener.backtester import FinancialBacktest

In [104]:
kse_return = FinancialBacktest(kse_price).ret()

D:\Udemy_Course\TradingDev\WiseData\FileOpener\backtester.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  ret.loc[:'1998-05-23'][ret.loc[:'1998-05-23'] > 1.08] = 1.08
C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\frame.py:3160: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._where(-key, value, inplace=True)
D:\Udemy_Course\TradingDev\WiseData\FileOpener\backtester.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/

## KOSPI 상따와 하따

### 기간: 2015.06.15 이후

#### 하한가가 난 종목을 10일간 보유할 시 수익률

In [105]:
kse_down_30 = kse_return.loc['2015-06-15':] < 0.71
kse_up_30 = kse_return.loc['2015-06-15':] > 1.29

## Cactch the signal

In [130]:
def test_materials(test_filter):
    
    test_data = {}
    
    for key in test_filter.columns:

        temp = test_filter[key][test_filter[key]==True].index

        if len(temp) == 0: 
            pass
        else:
            test_data[key] = temp
    
    return test_data

def backtest(sigal_f_stock, ret_data):
    
    ret = []
    
    for key, value in sigal_f_stock.items():
    
        for date in value:
            from_date = date + relativedelta(days=1)
            to_date = date + relativedelta(days=10)
            ret.append((ret_data[key].loc[from_date:to_date]).prod())
    
    return np.array(ret)

In [131]:
down_signal = test_materials(kse_down_30)
ret_signal = backtest(down_signal, kse_return)

In [148]:
win_rate = (ret_signal > 1).sum() / ret_signal.shape[0]

med_rt = (np.median(ret_signal) - 1)
avg_rt = (ret_signal.mean()-1)
min_rt = (ret_signal.min()-1)
max_rt = (ret_signal.max()-1)

## 하따 시 수익률

In [151]:
print("승률: ", '{:2.2%}'.format(win_rate))
print("매매 신호: ", ret_signal.shape[0])
print("중위수익률: ", '{:2.2%}'.format(med_rt))
print("평균 수익률: ", '{:2.2%}'.format(avg_rt))
print("최소 수익률: ", '{:2.2%}'.format(min_rt))
print("최대 수익률: ", '{:2.2%}'.format(max_rt))

승률:  27.62%
매매 신호:  105
중위수익률:  -10.00%
평균 수익률:  -13.59%
최소 수익률:  -93.06%
최대 수익률:  87.23%


In [134]:
up_signal = test_materials(kse_up_30)
up_ret_signal = backtest(up_signal, kse_return)

803

In [146]:
up_win_rate = (up_ret_signal > 1).sum() / up_ret_signal.shape[0]

up_med_rt = (np.median(up_ret_signal) - 1)
up_avg_rt = (up_ret_signal.mean()-1)
up_min_rt = (up_ret_signal.min()-1)
up_max_rt = (up_ret_signal.max()-1)

## 상따 시 수익률

In [150]:
print("매매 신호: ", up_ret_signal.shape[0])
print("승률:", '{:2.2%}'.format(up_win_rate))
print("평균 수익률: ", '{:2.2%}'.format(up_avg_rt))
print("중위 수익률: ", '{:2.2%}'.format(up_med_rt))
print("최소 수익률: ",'{:2.2%}'.format(up_min_rt))
print("최대 수익률: ",'{:2.2%}'.format(up_max_rt))

매매 신호:  803
승률: 42.22%
평균 수익률:  3.61%
중위 수익률:  -3.15%
최소 수익률:  -93.44%
최대 수익률:  310.26%
